<h1>0. Parametres</h1>

<h2>0.1. Cout Horaire</h2>

<h3>0.1.1. Profs</h3>

In [1]:
CH_Prof = {
    'Johanna':{
        'Niv1': 30,
        'Niv2': 30
    },
    'Merhan':{
        'Bodywork': 60,
        'LadyStyling': 60,
        'AtChor': 60,
        'Intensif': 55
    },
    'Samuel':{
        'Niv1': 55,
        'Niv2': 55,
        'AtChor': 55,
        'Intensif': 55
    }
    }

<h3>0.1.2. Salles</h3>

In [2]:
CH_Salle = {
    'Orpheon':{
        'Niv1': 17,
        'Niv2': 17
    },
    'StEtienne':{
        'Bodywork': 0,
        'LadyStyling': 0,
        'AtChor': 0,
        'Intensif': 0
    }
}

<h3>0.1.3. Assurance</h3>

In [3]:
CH_Assurance = {
    'Orpheon':{
        'Niv1': 0.51,
        'Niv2': 0.51,
        'Bodywork': 0.51,
        'LadyStyling': 0.51,
    }
}

<h2>0.2. Cout unitaire</h2>

<h3>0.2.1. Déplacement profs/cours

In [76]:
CU_Deplacement = {
    'Johanna': {
        'Niv1': 0,
        'Niv2': 0
    },
    'Merhan': {
        'Bodywork': 10,
        'LadyStyling': 10,
        'AtChor': 10,
        'Intensif': 10
    },
    'Samuel': {
        'Niv1': 0,
        'Niv2': 0,
        'AtChor': 0,
        'Intensif': 0
    }
}

<h2>0.3. Couts fixes</h2>

In [5]:
CF_Adhesion = {
    'Adhésion': 15
}

<h2>0.4. Taille des programmes</h2>

In [38]:
Cours = {
    'NbSeances':
    {
        'Niv1': 33,
        'Niv2': 33,
        'Bodywork': 15,
        'LadyStyling': 15,
        'AtChor': 10,
        'Intensif': 10
    },
    'DureeSeance':
    {
        'Niv1': 1.5,
        'Niv2': 1.5,
        'Bodywork': 1,
        'LadyStyling': 1.5,
        'AtChor': 2,
        'Intensif': 6
    }
}

<h2>0.5. Coefficients adhésions</h2>

<h3>0.5.1. Type d'adhésion</h3>

In [39]:
Coef_Adhesion = {
    '1Tri': [1/3, 10],
    '2Tri': [2/3, 5],
    '3Tri': [3/3, 0]
}

<h3>0.5.2. Réductions</h3>

In [40]:
Coef_Reduction = {
    2: 10,
    3: 15,
    4: 20
}

<h1>1. Import des librairies</h1>

In [41]:
import numpy as np
import pandas as pd

<h1>2. Générer des DataFrames</h1>

<h2>2.1. Génération des dataframes à partir des dictionnaires </h2>

In [77]:
DF_CH_Prof = pd.DataFrame(CH_Prof)
DF_CH_Salle = pd.DataFrame(CH_Salle)
DF_CH_Assurance = pd.DataFrame(CH_Assurance)
DF_CU_Deplacement = pd.DataFrame(CU_Deplacement)
DF_Cours = pd.DataFrame(Cours)
DF_Coef_Adhesion = pd.DataFrame(Coef_Adhesion)

<h2>2.2. Compléter les dataframes</h2>

In [78]:
DF_Cours['DureeTotal'] = DF_Cours['NbSeances'] * DF_Cours['DureeSeance']

<h2>2.3. Concaténer les dataframes</h2>

In [79]:
DF_CoutsHoraire = pd.concat(
    {
        'Prof': DF_CH_Prof,
        'Salle': DF_CH_Salle,
        'Assurance': DF_CH_Assurance,
    }, axis=1)

<h1>3. Calculer les budgets annuel</h2>

In [91]:
DF_Budget = pd.concat(
    [
        DF_CoutsHoraire.multiply(DF_Cours['DureeTotal'], axis=0),
        pd.concat({'Deplacement': DF_CU_Deplacement.multiply(DF_Cours['NbSeances'], axis=0)}, axis=1),
        
    ], axis=1)

In [92]:
DF_Budget

Prof                   Salle           Assurance Deplacement  \
            Johanna  Merhan  Samuel Orpheon StEtienne   Orpheon     Johanna   
Niv1         1485.0     NaN  2722.5   841.5       NaN    25.245         0.0   
Niv2         1485.0     NaN  2722.5   841.5       NaN    25.245         0.0   
Bodywork        NaN   900.0     NaN     NaN       0.0     7.650         NaN   
LadyStyling     NaN  1350.0     NaN     NaN       0.0    11.475         NaN   
AtChor          NaN  1200.0  1100.0     NaN       0.0       NaN         NaN   
Intensif        NaN  3300.0  3300.0     NaN       0.0       NaN         NaN   

                           
            Merhan Samuel  
Niv1           NaN    0.0  
Niv2           NaN    0.0  
Bodywork     150.0    NaN  
LadyStyling  150.0    NaN  
AtChor       100.0    0.0  
Intensif     100.0    0.0

In [137]:
BudgetAnnuelParCours = DF_Budget.sum(axis=1)
DF_Budget_Transpose = DF_Budget.T.reset_index(names=['Categorie','Description'])
BudgetAnnuelParCategorie = DF_Budget_Transpose.groupby('Categorie')[['Niv1','Niv2','Bodywork','LadyStyling','AtChor','Intensif']].sum().sum(axis=1)
BudgetAnnuelParDescription = DF_Budget.T.reset_index(names=['Categorie','Description']).groupby('Description')[['Niv1','Niv2','Bodywork','LadyStyling','AtChor','Intensif']].sum().sum(axis=1)
BudgetGlobal = DF_Budget.sum().sum()